


# 🛍️ Produkt Klasifikacija

Ova sveska prikazuje primer klasifikacije proizvoda u kategorije pomoću **TF-IDF** i **Logistic Regression** modela. Cilj ovog notebook-a je razviti model mašinskog učenja koji predviđa kategoriju proizvoda na osnovu naziva.  
Ovaj model omogućava automatsku klasifikaciju proizvoda u online trgovini, čime se smanjuje vreme unosa i mogućnost greške.

In [ ]:
# Učitavanje potrebnih biblioteka
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle


## 1. Učitavanje i pregled podataka


In [ ]:
# Učitavanje CSV fajla sa punim path-om
file_path = r"C:\Users\Administrator\Desktop\IMLP4_TASK_03-products.csv"
df = pd.read_csv(file_path)

# Pregled prvih 5 redova
df.head()



## 2. Čišćenje podataka

- Provera praznih vrednosti
- Standardizacija naziva kolona


In [ ]:


# Standardizacija naziva kolona: uklanjanje razmaka, mala slova, zamena specijalnih karaktera
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()

# Prikaz prvih nekoliko redova i kolona
print("Nazivi kolona:", df.columns)
print(df.head())

# Provera praznih vrednosti
print("\nBroj praznih vrednosti po koloni:")
print(df.isnull().sum())

# Uklanjanje redova sa nedostajućim nazivom proizvoda ili kategorijom

df = df.dropna(subset=['product_title', 'category_label'])

# Prikaz broja redova nakon čišćenja
print("\nBroj redova nakon uklanjanja praznih vrednosti:", len(df))


## 3. Feature engineering

- Dužina naslova proizvoda
- Broj reči
- Prisustvo brojeva u nazivu


In [ ]:
# Kreiranje dodatnih feature-a na osnovu naziva proizvoda
df['title_length'] = df['product_title'].apply(len)
df['word_count'] = df['product_title'].apply(lambda x: len(str(x).split()))
df['has_number'] = df['product_title'].apply(lambda x: any(char.isdigit() for char in str(x)))
df['has_uppercase'] = df['product_title'].apply(lambda x: any(char.isupper() for char in str(x)))

# Prikaz prvih nekoliko redova sa novim feature-ima
df[['product_title', 'title_length', 'word_count', 'has_number', 'has_uppercase']].head()



## 4. Priprema podataka za model

- TF-IDF vektorizacija naziva proizvoda
- Kodiranje kategorija


In [ ]:
# Vektorizacija naziva proizvoda
tfidf = TfidfVectorizer(max_features=5000)
X_text = tfidf.fit_transform(df['product_title'])

# Kombinovanje sa dodatnim feature-ima
X_additional = df[['title_length', 'word_count', 'has_number']].astype(float)
from scipy.sparse import hstack
X = hstack([X_text, X_additional])

# Kodiranje ciljnih kategorija
le = LabelEncoder()
y = le.fit_transform(df['category_label'])

# Podela na trening i test skup
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



## 5. Treniranje i evaluacija modela

- Uporedićemo Naive Bayes i Logistic Regression



In [ ]:
# Treniranje Naive Bayes modela
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)
print("Naive Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb, target_names=le.classes_))

# Treniranje Logistic Regression modela
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr, target_names=le.classes_))


## 6. Sačuvaj finalni model i vektorizator


In [ ]:
# Čuvanje Logistic Regression modela i TF-IDF vektorizatora
with open('final_model.pkl', 'wb') as f:
    pickle.dump(lr_model, f)

with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)


## 7. Interaktivno testiranje modela


In [ ]:
# Učitavanje modela, vektorizatora i label encoder-a
with open('final_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
with open('tfidf_vectorizer.pkl', 'rb') as f:
    loaded_tfidf = pickle.load(f)
with open('label_encoder.pkl', 'rb') as f:
    loaded_le = pickle.load(f)

# Funkcija za predikciju kategorije
def predict_category(product_title):
    x_text = loaded_tfidf.transform([product_title])
    x_additional = np.array([[len(product_title), len(product_title.split()), any(char.isdigit() for char in product_title)]])
    from scipy.sparse import hstack
    X_final = hstack([x_text, x_additional])
    pred = loaded_model.predict(X_final)
    return loaded_le.inverse_transform(pred)[0]

# Testiranje
test_titles = [
    "iphone 7 32gb gold",
    "olympus e m10 mark iii geh use silber",
    "kenwood k20mss15 solo",
    "bosch wap28390gb 8kg 1400 spin",
    "smeg sbs8004po"
]

for title in test_titles:
    print(f"{title} --> {predict_category(title)}")


## 8. Zaključak

- Model može automatski predvideti kategorije proizvoda sa velikom preciznošću.
- Dodatne karakteristike naslova (dužina, broj reči, prisustvo brojeva) poboljšavaju performanse.
- Logistic Regression je finalni model zbog bolje tačnosti i stabilnosti.
- Model je sačuvan u `.pkl` formatu i može se koristiti za interaktivno predviđanje novih proizvoda.
